In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
data = {
    'Fe':[60,62,58,61,59],
    'SiO2':[5,4.5,6,5.2,5.5],
    'Al2O3':[2,1.8,2.2,2,1.9],
    'CaO':[1,1.2,0.8,1,1.1],
    'Limestone':[10,12,9,11,10],
    'Dolomite':[5,6,4,5,5],
    'BlastTemp':[1200,1180,1210,1190,1205],
    'CokeRate':[450,430,460,440,445],
    'SiO2_slag':[28,27,29,28,28],
    'CaO_slag':[38,39,37,38,38],
    'Basicity':[1.36,1.44,1.28,1.36,1.36]
}
df = pd.DataFrame(data)
df

,Fe,SiO2,Al2O3,CaO,Limestone,Dolomite,BlastTemp,CokeRate,SiO2_slag,CaO_slag,Basicity
0,60,5.0,2.0,1.0,10,5,1200,450,28,38,1.36
1,62,4.5,1.8,1.2,12,6,1180,430,27,39,1.44
2,58,6.0,2.2,0.8,9,4,1210,460,29,37,1.28
3,61,5.2,2.0,1.0,11,5,1190,440,28,38,1.36
4,59,5.5,1.9,1.1,10,5,1205,445,28,38,1.36


In [3]:
X = df[['Fe','SiO2','Al2O3','CaO','Limestone','Dolomite','BlastTemp','CokeRate']]
y = df[['SiO2_slag','CaO_slag','Basicity']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [4]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)**0.5
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}\n")


MAE: 0.70
RMSE: 0.83



In [5]:
new_sample = [[61,5,2,1,11,5,1190,440]]
predicted_slag = model.predict(new_sample)[0]
print("Predicted Slag Composition:")
print(f"SiO2: {predicted_slag[0]:.2f}%")
print(f"CaO: {predicted_slag[1]:.2f}%")
print(f"Basicity (CaO/SiO2): {predicted_slag[2]:.2f}\n")


Predicted Slag Composition:
SiO2: 28.01%
CaO: 37.99%
Basicity (CaO/SiO2): 1.36



/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [6]:
def slag_solutions(SiO2, CaO, basicity, target_basicity=1.4):
    suggestions = []
    # basicity
    if basicity<target_basicity:
        suggestions.append("Increase flux addition (Limestone/Dolomite) to raise basicity.")
    elif basicity>target_basicity:
        suggestions.append("Reduce flux addition to lower basicity.")
    # SiO2 content
    if SiO2>30:
        suggestions.append("Consider adding more CaO flux to reduce SiO2 content.")
    elif SiO2<25:
        suggestions.append("Slightly reduce CaO addition to increase SiO2 if needed.")
    # CaO content
    if CaO<35:
        suggestions.append("Increase limestone/dolomite addition to increase CaO.")
    elif CaO>40:
        suggestions.append("Reduce flux addition to lower CaO.")
    if not suggestions:
        suggestions.append("Slag composition is within optimal range.")
    return suggestions
solutions = slag_solutions(predicted_slag[0], predicted_slag[1], predicted_slag[2])
print("Suggested Corrective Actions:")
for s in solutions:
    print("-", s)

Suggested Corrective Actions:
- Increase flux addition (Limestone/Dolomite) to raise basicity.


In [7]:
data2 = {
    'Fe':[60,62,78,91,59],
    'SiO2':[5,4.5,6,5.2,5.5],
    'Al2O3':[2,1.9,2.6,2.1,1.3],
    'CaO':[1,1.2,0.8,1,1.1],
    'Limestone':[10,12,9,11,10],
    'Dolomite':[5,6,4,5,5],
    'BlastTemp':[1210,1160,1210,1145,1210],
    'CokeRate':[452,437,465,444,445],
    'SiO2_slag':[30,39,29,28,28],
    'CaO_slag':[38,37,37,32,38],
    'Basicity':[1.36,1.44,1.28,1.36,1.36]
}


In [8]:
import pickle
pickle.dump(RandomForestRegressor(n_estimators=100, random_state=42), open('model.pkl', 'wb'))
model=pickle.load(open('model.pkl', 'rb'))
model.fit(X_train, y_train)
df = pd.DataFrame(data2)
predictions = model.predict(df[['Fe','SiO2','Al2O3','CaO','Limestone','Dolomite','BlastTemp','CokeRate']])
pred_df = pd.DataFrame(predictions, columns=['SiO2_slag','CaO_slag','Basicity'])
pred_df['Solutions'] = pred_df.apply(
    lambda row: " | ".join(slag_solutions(row['SiO2_slag'], row['CaO_slag'], row['Basicity'])), axis=1
)
data2_df = pd.DataFrame(data2)
final_df = pd.concat([data2_df, pred_df], axis=1)
final_df.to_csv("slag_predictions_with_solutions.csv", index=False)
print(final_df)
import joblib
joblib.dump(model, "slag_model.pkl")

   Fe  SiO2  Al2O3  CaO  Limestone  Dolomite  BlastTemp  CokeRate  SiO2_slag  \
0  60   5.0    2.0  1.0         10         5       1210       452         30   
1  62   4.5    1.9  1.2         12         6       1160       437         39   
2  78   6.0    2.6  0.8          9         4       1210       465         29   
3  91   5.2    2.1  1.0         11         5       1145       444         28   
4  59   5.5    1.3  1.1         10         5       1210       445         28   

   CaO_slag  Basicity  SiO2_slag  CaO_slag  Basicity  \
0        38      1.36      28.06     37.94    1.3552   
1        37      1.44      28.01     37.99    1.3592   
2        37      1.28      28.54     37.46    1.3168   
3        32      1.36      28.01     37.99    1.3592   
4        38      1.36      28.05     37.95    1.3560   

                                           Solutions  
0  Increase flux addition (Limestone/Dolomite) to...  
1  Increase flux addition (Limestone/Dolomite) to...  
2  Increase flux 

['slag_model.pkl']